# Run simple urban extent calculation based on 1km gridded population data

In [1]:
import sys
import os
import rasterio

import geopandas as gpd

import GOST_Urban.UrbanRaster as urban

In [2]:
# Define input population raster
tutorial_folder = "../../data/tutorial_data"
aoi_file = os.path.join(tutorial_folder, "AOI.geojson")
pop_file = os.path.join(tutorial_folder, "wp_2020_1k_AOI.tif")

# Define output files
out_urban = os.path.join(tutorial_folder, "urban_extents.geojson")
out_hd_urban = os.path.join(tutorial_folder, "hd_urban_extents.geojson")

inAOI = gpd.read_file(aoi_file)

# Shouldn't need to execute this unless you change your AOI;
#    you will need to find a global population to extract from
if not os.path.exists(pop_file):
    sys.path.append("../../../gostrocks/src")
    import GOSTRocks.rasterMisc as rMisc

    global_population = "/path/to/global/pop_layer/ppp_2020_1km_Aggregated.tif"
    inR = rasterio.open(global_population)
    rMisc.clipRaster(inR, inAOI, pop_file)

inR = rasterio.open(pop_file)

Cannot find header.dxf (GDAL_DATA is not defined)


In [4]:
# Initiate the urban calculator
urban_calculator = urban.urbanGriddedPop(inR)
# urban_calculator.calculateUrban?

In [ ]:
# Extract the urban extents (minimum density 300/km2, minimum total population 5000)
urban_extents = urban_calculator.calculateUrban(
    densVal=300, totalPopThresh=5000, smooth=False, queen=False, verbose=True
)

In [5]:
# Extract the high density urban extents (minimum density 1500/km2, minimum total population 50000)
hd_urban_extents = urban_calculator.calculateUrban(
    densVal=1500,
    totalPopThresh=50000,
    smooth=True,
    queen=True,  # high density extents use queen's case contiguity, and
    verbose=True,
)  # High density extents have hole smoothing applied.

15:11:38	: Read in urban data
15:11:38	: Creating Shape 0
15:11:44	: Creating Shape 1000


C:\wbg\Anaconda3\envs\test\Lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\wbg\Anaconda3\envs\test\Lib\site-packages\GOST_Urban\UrbanRaster.py:322: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  xxGeom['geometry'] = xxGeom.buffer((popRaster.res[0] / 2))


In [ ]:
urban_extents.to_file(out_urban, driver="GeoJSON")
hd_urban_extents.to_file(out_hd_urban, driver="GeoJSON")